## Parallel Programming with Python
Angel David Sansores Cruz\
Universidad Politécnica de Yucatán\
2109139@upy.edu.mx

![Instructions](instructions.png)


## 1. Write a program in Pytho nwhich solves the program without any parallelization.

## Non-parallel Computation of π

This section demonstrates how to compute an approximation of π using a non-parallel method. We'll calculate the area under the curve of a quarter circle using the Riemann sum approach. The computation is done sequentially in a single process. Here's a breakdown of the steps:

1. Calculate the width of each interval (`dx`).
2. Compute the sum of the areas of rectangles under the curve using the formula `sqrt(1 - x^2)`.
3. Multiply the total by 4 to estimate π.

We'll also measure the time taken to perform this calculation to compare it with parallel methods.


In [1]:
import numpy as np
import time

In [2]:
def compute_pi_sequential(N):
    dx = 1.0 / N
    total = sum(np.sqrt(1 - (i * dx) ** 2) for i in range(N))
    return 4 * total * dx

N = 1000000
start_time = time.time()  # Start timing
pi_approx_sequential = compute_pi_sequential(N)
end_time = time.time()  # End timing

print(f"Sequential π approximation: {pi_approx_sequential}")
print(f"Time taken: {end_time - start_time:.4f} seconds")



Sequential π approximation: 3.141594652413976
Time taken: 1.2785 seconds


## 2. Write a program in Python which usesparallel computing via multiprocessingto solve the problem.

## Parallel Computation of π using Multiprocessing

In this section, we use Python's `multiprocessing` library to parallelize the computation of π. This method splits the task across multiple processes, which can run on separate cores:

1. We define a function `f` that computes the area for a given interval.
2. We use a pool of worker processes to compute these areas in parallel.
3. We aggregate the results from all processes to compute the final approximation of π.

The function `test_different_Ns` tests the performance for different values of N and prints the results, including the time taken for each computation.


In [3]:
from multiprocessing import Pool
import numpy as np
import time

In [4]:
def f(i, dx):
    x = i * dx
    return np.sqrt(1 - x**2) * dx

def compute_pi_multiprocessing(N, num_processes):
    dx = 1.0 / N
    with Pool(num_processes) as pool:
        result = pool.starmap(f, [(i, dx) for i in range(N)])
    return 4 * sum(result)

def test_different_Ns():
    test_values = [10000, 100000, 1000000]  # Example values for N
    num_processes = 4  # Example number of processes
    results = []
    
    for N in test_values:
        start_time = time.time()
        pi_approx = compute_pi_multiprocessing(N, num_processes)
        end_time = time.time()
        
        execution_time = end_time - start_time
        results.append((N, pi_approx, execution_time))
        
        print(f"N = {N}: Multiprocessing π approximation = {pi_approx:.6f}, Time taken = {execution_time:.2f} seconds")

    return results

In [ ]:
# Execute the test function
if __name__ == '__main__':
    test_results = test_different_Ns()

## 3. Write a program in Python which uses distributed parallel computing via mi4pyto solve the problem.

## Distributed Parallel Computation of π using MPI

For distributed parallel computing, we utilize `mpi4py`, a Python wrapper for the MPI interface. This approach is suitable for running on clusters or multi-core systems in a distributed manner:

1. Each process computes a portion of the π approximation independently.
2. We utilize `np.linspace` to calculate the x-values that each process should handle.
3. Local results are aggregated using the MPI `reduce` method to compute the global sum, which is then used to estimate π.

Timing is performed individually in each process, but only the root process (rank 0) outputs the final time and result.


In [1]:
!pip install mpi4py

In [2]:
from mpi4py import MPI
import numpy as np
import time

In [3]:
def compute_pi(N):
    # Initialize MPI environment
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Compute number of intervals handled by each process
    local_n = N // size + (rank < N % size)
    start = rank * local_n
    end = start + local_n

    start_time = time.time()  # Start timing on each process

    # Calculate local integral
    x = np.linspace(start/N, end/N, local_n, endpoint=False)
    local_sum = np.sum(np.sqrt(1 - x**2)) * (1.0 / N)

    # Gather all local integrals to the root process
    pi_approx = 4 * comm.reduce(local_sum, op=MPI.SUM, root=0)

    end_time = time.time()  # End timing on each process

    # Root process prints the result
    if rank == 0:
        print("Pi with MPI:", pi_approx)
        print(f"Time taken (root process): {end_time - start_time:.4f} seconds")

if __name__ == "__main__":
    N = 10_500_000
    compute_pi(N)

Pi with MPI: 3.141592844031421
Time taken (root process): 0.2581 seconds
